In [15]:
# --- 🔧 载入环境 ---
from pathlib import Path
from Backend.Inference import ModelWrapper, ModelConfig  # ← 改成你项目的模块路径


In [16]:
# 获取当前目录
base_dir = Path(__file__).parent if "__file__" in locals() else Path(".")

# 指向 Backend 目录
backend_dir = base_dir / "Backend"

# 模型文件路径
model_path = backend_dir / "saved_models" / "model_20251007_123355.h5"
label_path = backend_dir / "saved_models" / "id2label_20251007_123355.pkl"

print(model_path)
print(label_path)

# ✅ 初始化配置对象
config = ModelConfig()

# ✅ 创建模型包装器实例
wrapper = ModelWrapper(
    model_path=model_path,
    label_path=label_path,
    config=config
)

print("✅ 模型加载成功！")

2025-11-07 00:06:58,667 - Backend.Inference - INFO - 🚀 开始加载模型...


Backend\saved_models\model_20251007_123355.h5
Backend\saved_models\id2label_20251007_123355.pkl


2025-11-07 00:06:59,160 - absl - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2025-11-07 00:06:59,179 - Backend.Inference - INFO - 🔥 加载 MiniLM 编码器...
2025-11-07 00:06:59,180 - Backend.Inference - INFO -    从 Hugging Face 加载: sentence-transformers/all-MiniLM-L6-v2
2025-11-07 00:06:59,184 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-11-07 00:06:59,184 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-11-07 00:07:02,677 - Backend.Inference - INFO - ✅ MiniLM 编码器加载成功
2025-11-07 00:07:02,677 - Backend.Inference - INFO - 🔥 加载 BART 编码器...
2025-11-07 00:07:02,678 - Backend.Inference - INFO -    从 Hugging Face 加载: facebook/bart-large
2025-11-07 00:07:02,678 - Backend.Inference - INFO -    使用默认缓存: ~/.cache/huggingface/
2025-11-07 00:07:02,682 - sentence_trans

✅ 模型加载成功！


In [21]:
def diagnose_input(text):
    print(f"\n🧩 输入句子: {text}")

    # 1️⃣ 生成 BART 向量
    bart_emb = SentenceTransformer('facebook/bart-large').encode([text])
    bart_emb = bart_emb.reshape((1, 1, bart_emb.shape[1]))  # (1, 1, 1024)

    # 2️⃣ 生成 RoBERTa 向量
    roberta_emb = RepresentationModel(model_type="roberta", model_name="roberta-base", use_cuda=False)
    roberta_emb = roberta_emb.encode_sentences([text], combine_strategy="mean")
    roberta_emb = roberta_emb.reshape((1, 1, roberta_emb.shape[1]))  # (1, 1, 768)

    print(f"📏 BART shape: {bart_emb.shape}, RoBERTa shape: {roberta_emb.shape}")

    # 3️⃣ 模型预测
    preds = model.predict([bart_emb, roberta_emb])
    label_id = int(np.argmax(preds, axis=1)[0])
    print(f"🔮 模型预测标签: {id2label[label_id]}")


In [22]:
# --- 🧠 测试 ---
diagnose_input("Mount Everest is the tallest mountain on Earth")
diagnose_input("The moon is made of cheese")
diagnose_input("Donald Trump is the current president of the USA")


🧩 输入句子: Mount Everest is the tallest mountain on Earth


NameError: name 'SentenceTransformer' is not defined